## 部署和使用CPMBee
进行中文问答
部署后失败，不能推理

### SageMaker  Endpoint 部署CPM-Bee


#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [1]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

接下来，我们使用Sagemaker进行模型部署。

In [16]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = 'inference-cpmbee.py',
    source_dir = './code-cpmbee',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


我们使用了bring your own script 的方式部署模型，通过提供一个dummy的模型文件（model.tar.gz），在 *code/inference-chatglm.py* python脚本里面，部署了模型：
```python
def model_fn(model_dir):
    model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
    return model
```


然后部署该模型为 Sagemaker endpoint

In [17]:

# deploy as sagemaker endpoint
endpoint_name = 'mt-cpmbee-g4dn'
instance_type = 'ml.g4dn.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

------------!

In [32]:
from sagemaker import serializers, deserializers

del_predictor = sagemaker.Predictor(
    endpoint_name="mt-cpmbee-5b-g4dn", 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer())
del_predictor.delete_model()
del_predictor.delete_endpoint()
# predictor.delete_model()
# predictor.delete_endpoint()

In [2]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name="mt-cpmbee-g4dn",
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer())

In [18]:
inputs= [
    {"inputs": {"input": "写一首关于交通信号灯的诗", "<ans>": ""}, "parameters": {"max_new_tokens": 1000}},
    {"inputs": {"input": "陨石为什么总能落在陨石坑里?", "<ans>": ""}, "parameters": {"max_new_tokens": 1000}},
    {"inputs": {"input": "为什么爸妈结婚没叫我参加婚礼?", "<ans>": ""}, "parameters": {"max_new_tokens": 1000}}
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["inputs"]["input"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["inputs"]["input"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["inputs"]["input"], "\n回答:\n", response["outputs"])



问题:  写一首关于交通信号灯的诗 
回答:
 [{'input': '写一首关于交通信号灯的诗', '<ans>': '写一首关于交通信号灯的诗'}]


问题:  陨石为什么总能落在陨石坑里? 
回答:
 [{'input': '陨石为什么总能落在陨石坑里?', '<ans>': 'Why do meteorites always fall in craters?'}]


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
 [{'input': '为什么爸妈结婚没叫我参加婚礼?', '<ans>': '为什么爸妈结婚没叫我参加婚礼？'}]


In [21]:
%%time
import json

data_list = [
    {"input": "今天天气是真的", "prompt": "往后写两句话", "<ans>": ""},
    {"input": "北京市气象台提示，4月12日午后偏南风加大，阵风可达6级左右，南下的沙尘可能伴随回流北上进京，外出仍需注意<mask_0>，做好健康防护。天津市气象台也提示，受<mask_1>影响，我市4月12日有浮尘天气，PM10浓度<mask_2>。请注意关好门窗，老人儿童尽量减少户外活动，外出注意带好<mask_3>。” ","<ans>":{"<mask_0>":"","<mask_1>":"","<mask_2>":"","<mask_3>":""}},
]
# result = model.generate({"input": "今天天气不错，", "<ans>": ""}, tokenizer)
result = predictor.predict({ 
                    "inputs" : data_list,
                    "parameters": {"max_new_tokens": 50, "repetition_penalty": 1.1, "temperature": 0.5}
                })
for res in result["outputs"]:
    print(res)

{'input': '今天天气是真的', 'prompt': '往后写两句话', '<ans>': '好啊，阳光明媚，万里无云。\n今天天气是真的好啊，阳光明媚，万里无云。'}
{'input': '北京市气象台提示，4月12日午后偏南风加大，阵风可达6级左右，南下的沙尘可能伴随回流北上进京，外出仍需注意<mask_0>，做好健康防护。天津市气象台也提示，受<mask_1>影响，我市4月12日有浮尘天气，PM10浓度<mask_2>。请注意关好门窗，老人儿童尽量减少户外活动，外出注意带好<mask_3>。” ', '<ans>': {'<mask_0>': '防风', '<mask_1>': '沙尘天气', '<mask_2>': '较高', '<mask_3>': '口罩、护目镜等防护用品'}}
CPU times: user 4.04 ms, sys: 1e+03 ns, total: 4.04 ms
Wall time: 2.09 s


In [3]:
def create_input():
    return {
        "inputs" : {
            "input": "",
            "prompt": "",
            "question": "",
            "<ans>": ""
        }, 
        "parameters": { "max_new_tokens": 2048, "temperature": 0.01 }
    }
    
inputs = create_input()

In [4]:
inputs["inputs"]["input"] = ""
inputs["inputs"]["prompt"] = ""
inputs["inputs"]["question"] = "如果我在空中垂直抛球，球最初向哪个方向行进？"
predictor.predict(inputs)

{'outputs': [{'input': '',
   'prompt': '',
   'question': '如果我在空中垂直抛球，球最初向哪个方向行进？',
   '<ans>': '垂直'}]}

In [5]:
%%time
inputs = create_input()
inputs["inputs"]["question"] = "萨莉把一个小球放到篮子里，然后用布将篮子盖起来后离开。安娜在萨莉走后，将小球从篮中取出放进身边的盒子里。过一会儿，萨莉回来，萨莉会到哪里去找小球呢？"
predictor.predict(inputs)

CPU times: user 0 ns, sys: 3.53 ms, total: 3.53 ms
Wall time: 359 ms


{'outputs': [{'input': '',
   'prompt': '',
   'question': '萨莉把一个小球放到篮子里，然后用布将篮子盖起来后离开。安娜在萨莉走后，将小球从篮中取出放进身边的盒子里。过一会儿，萨莉回来，萨莉会到哪里去找小球呢？',
   '<ans>': '篮子'}]}

In [28]:
%%time

inputs = create_input()
inputs1 = {
    "input": "同源结构经常被引用作为自然选择过程的证据。",
    "prompt":"选择题",
    "question:": "以下所有的结构都是同源结构，除了:",
    "options": {
      "<option_0>": "(A) 鸟的翅膀和蝙蝠的翅膀", 
      "<option_1>": "(B) 鲸鱼的鳍和人的手臂",
      "<option_2>": "(C) 海豚的胸鳍和海豹的鳍",
      "<option_3>": "(D) 昆虫的前腿和狗的前肢"
    },
    "<ans>": ""
}
inputs["inputs"] = inputs1
predictor.predict(inputs)

CPU times: user 1.51 ms, sys: 3.02 ms, total: 4.53 ms
Wall time: 368 ms


{'outputs': [{'input': '同源结构经常被引用作为自然选择过程的证据。',
   'prompt': '选择题',
   'question:': '以下所有的结构都是同源结构，除了:',
   'options': {'<option_0>': '(A) 鸟的翅膀和蝙蝠的翅膀',
    '<option_1>': '(B) 鲸鱼的鳍和人的手臂',
    '<option_2>': '(C) 海豚的胸鳍和海豹的鳍',
    '<option_3>': '(D) 昆虫的前腿和狗的前肢'},
   '<ans>': '<option>'}]}

In [30]:
%%time
inputs = create_input()
inputs1 = {
    "input": "父母都希望自己的孩子诚实、勇敢、有礼貌。要想让孩子成为这样的人，父母首先得从自己做起，要是连自己都做不到，又怎能要求孩子做到呢？", 
    "options": {
      "<option_0>": "少提要求", 
      "<option_1>": "降低标准",
      "<option_2>": "自己先做好",
      "<option_3>": "让孩子拿主意"
    }, 
    "question": "教育孩子时，父母应该：", 
    "<ans>": ""
  }
# inputs1 = {"input": "今天天气真好，", "prompt":"选择题", "question": "今天天气怎么样", "options": {"<option_0>": "好", "<option_1>": "坏"}, "<ans>":""}
inputs["inputs"] = inputs1
predictor.predict(inputs)


CPU times: user 4.12 ms, sys: 0 ns, total: 4.12 ms
Wall time: 353 ms


{'outputs': [{'input': '父母都希望自己的孩子诚实、勇敢、有礼貌。要想让孩子成为这样的人，父母首先得从自己做起，要是连自己都做不到，又怎能要求孩子做到呢？',
   'options': {'<option_0>': '少提要求',
    '<option_1>': '降低标准',
    '<option_2>': '自己先做好',
    '<option_3>': '让孩子拿主意'},
   'question': '教育孩子时，父母应该：',
   '<ans>': '<option_1>'}]}

In [6]:
%%time
inputs = create_input()
inputs1 = {
    "input": "2022年世界杯的冠军是阿根廷队伍，梅西是MVP", 
    "<ans>": {"人名":"","国家名":""}
  }
# inputs1 = {"input":"在司法部工作的小楠说，今天北京天气不错","<ans>":{"人名":"","地名":"","机构名": ""}}
inputs["inputs"] = inputs1
predictor.predict(inputs)


CPU times: user 989 µs, sys: 3.43 ms, total: 4.42 ms
Wall time: 461 ms


{'outputs': [{'input': '2022年世界杯的冠军是阿根廷队伍，梅西是MVP',
   '<ans>': {'人名': '梅西', '国家名': '阿根廷'}}]}

In [41]:
%%time
text = """已知内容:
Q：水光针多久打一次呢?效果能维持多久?
A：水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q:水光针做完一次可以维持多久?
A:对于肤质干燥且疏于保养的人而言，其效果大约能维持3个月左右，保养越好，效果维持时间相应越长。具体来说，水光针注射效果与接受注射的次数有关，连续进行三次注射后，效果可以维持1—2年左右，但根据个人体质、肤质、生活习惯等不同，具体的维持时间也会有所差异。

Q:水光针怎样打效果更好?
A:水光针的话，我认为用机器打，特别是这个34G的16针头去操作的话，效果一定会更好，然后再用到好的产品，那么就事半功倍了，然后术后的话可以多敷修复的面膜。然后在就是持续的去打，那么可以，达到一个一加一大于二的效果。

Q:水光针多久打一次呢?效果能维持多久?
A:水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q:脸部、肚子溶脂针多久见效?
A:一般打完溶脂针的话就是五到七天会有效果，但是效果的显著程度因人而异，有的人打一次溶脂针效果就比较明显，但是有的人需要多次注射才会有明显的效果。除此之外，打完溶脂针以后，自身对手术部位的护理也会影响到溶脂针的见效速度与效果的好坏。
在打完溶脂针以后，要尽量的保持注射部位的清洁,在一天之内不能沾水，恢复期间也要避免外界的污染，更不能化妆，不要在这个注射部位涂抹外用的一些药物,以免刺激到注射的部位，影响溶脂针的一个效果，见效的时间也会因此而延长。
想要溶脂针的效果更好的,见效更快的话，多喝水也是可以起到一些的作用，虽然说打完溶脂针以后不能沾水，但是我们可以多喝水，最好是每天饮用一升以上的水，不仅可以促进身体的新陈代谢，水循环，对注射部位的恢复也会起到辅助的作用。
溶脂针就是注射一次,就有一次的效果,一般的话是按照疗程来注射，效果是最好的,疗程一般是六到八周左右，每两到三周就可以进行二次注射。
正常是第一次注射就能看到效果，第二次注射的时候的效果更加明显，第三次注射的时候就起到了一个巩固的作用。

<sep>
"""
inputs1 = {
    "input": text,
    # "prompt": "问答",
    "prompt":"使用已知内容，以及对话记录，简单直接的回答最后的问题。如果你从已知信息中无法知道答案，就直接返回'根据已知信息无法回答该问题.', 不要使用已有知识，不要编造答案。",
    "question": "水光针多久打一次呢?",
    "<ans>":""
}

inputs = create_input()
inputs["inputs"] = inputs1
predictor.predict(inputs)


CPU times: user 4.28 ms, sys: 89 µs, total: 4.37 ms
Wall time: 2.88 s


{'outputs': [{'input': '已知内容:\nQ：水光针多久打一次呢?效果能维持多久?\nA：水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。\n如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。\n水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月\n左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。\n\nQ:水光针做完一次可以维持多久?\nA:对于肤质干燥且疏于保养的人而言，其效果大约能维持3个月左右，保养越好，效果维持时间相应越长。具体来说，水光针注射效果与接受注射的次数有关，连续进行三次注射后，效果可以维持1—2年左右，但根据个人体质、肤质、生活习惯等不同，具体的维持时间也会有所差异。\n\nQ:水光针怎样打效果更好?\nA:水光针的话，我认为用机器打，特别是这个34G的16针头去操作的话，效果一定会更好，然后再用到好的产品，那么就事半功倍了，然后术后的话可以多敷修复的面膜。然后在就是持续的去打，那么可以，达到一个一加一大于二的效果。\n\nQ:水光针多久打一次呢?效果能维持多久?\nA:水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。\n如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。\n水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月\n左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。\n\nQ:脸部、肚子溶脂针多久见效?\nA:一般打完溶脂针的话就是五到七天会有效果，但是效果的显著程度因人而异，有的人打一次溶脂针效果就比较明显，但是有的人需要多次注射才会有明显的效果。除此之外，打完溶脂针以后，自身对手术部位的护理也会影响到溶脂针的见效速度与效果的好坏。\n在打完溶脂针以后，要尽量的保持注射部位的清洁,在一天之内不能沾水，恢复期间也要避免外界的污染，更不能化妆，不要在这个注射部位涂抹外用的一些药物,以免刺激到注射的部位，影

In [39]:
%%time
text = """已知内容:
Q:水光针多久打一次呢?效果能维持多久?
A:水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q：水光针多久打一次呢?效果能维持多久?
A：水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。
如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。
水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月
左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。

Q:水光针做完一次可以维持多久?
A:对于肤质干燥且疏于保养的人而言，其效果大约能维持3个月左右，保养越好，效果维持时间相应越长。具体来说，水光针注射效果与接受注射的次数有关，连续进行三次注射后，效果可以维持1—2年左右，但根据个人体质、肤质、生活习惯等不同，具体的维持时间也会有所差异。

Q:水光针怎样打效果更好?
A:水光针的话，我认为用机器打，特别是这个34G的16针头去操作的话，效果一定会更好，然后再用到好的产品，那么就事半功倍了，然后术后的话可以多敷修复的面膜。然后在就是持续的去打，那么可以，达到一个一加一大于二的效果。

Q:脸部、肚子溶脂针多久见效?
A:一般打完溶脂针的话就是五到七天会有效果，但是效果的显著程度因人而异，有的人打一次溶脂针效果就比较明显，但是有的人需要多次注射才会有明显的效果。除此之外，打完溶脂针以后，自身对手术部位的护理也会影响到溶脂针的见效速度与效果的好坏。
在打完溶脂针以后，要尽量的保持注射部位的清洁,在一天之内不能沾水，恢复期间也要避免外界的污染，更不能化妆，不要在这个注射部位涂抹外用的一些药物,以免刺激到注射的部位，影响溶脂针的一个效果，见效的时间也会因此而延长。
想要溶脂针的效果更好的,见效更快的话，多喝水也是可以起到一些的作用，虽然说打完溶脂针以后不能沾水，但是我们可以多喝水，最好是每天饮用一升以上的水，不仅可以促进身体的新陈代谢，水循环，对注射部位的恢复也会起到辅助的作用。
溶脂针就是注射一次,就有一次的效果,一般的话是按照疗程来注射，效果是最好的,疗程一般是六到八周左右，每两到三周就可以进行二次注射。
正常是第一次注射就能看到效果，第二次注射的时候的效果更加明显，第三次注射的时候就起到了一个巩固的作用。

<sep>问题: 水光针多久打一次呢?
<sep>回答：一个月
"""
inputs1 = {
    "input": text,
    # "prompt": "问答",
    "prompt":"使用已知内容，以及对话记录，简单直接的回答最后的问题。如果你从已知信息中无法知道答案，就直接返回'根据已知信息无法回答该问题.', 不要使用已有知识，不要编造答案。",
    "question": "如果加了肉毒呢?",
    "<ans>":""
}

inputs = create_input()
inputs["inputs"] = inputs1
predictor.predict(inputs)



CPU times: user 4.26 ms, sys: 64 µs, total: 4.32 ms
Wall time: 2.22 s


{'outputs': [{'input': '已知内容:\nQ:水光针多久打一次呢?效果能维持多久?\nA:水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。\n如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。\n水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月\n左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。\n\nQ：水光针多久打一次呢?效果能维持多久?\nA：水光针一般建议一个月左右打一次,大概就是28天,因为刚好皮肤的周期是这么久。\n如果是第一次打水光，吸收比较快，也一些专家建议第一次和第二次间隔半个月,二十天左右就可以打第二次,之后在打就间隔一个月左右。\n水光的功效主要看你加了什么成分，正常来说的话，一般的营养成分大概是一到两个月。可以维持,如果里面加了肉毒素的话，效果可以维持3个月\n左右,所以像水光如果你不加肉毒素，是每个月都可以操作注射的，越打皮肤会越好。\n\nQ:水光针做完一次可以维持多久?\nA:对于肤质干燥且疏于保养的人而言，其效果大约能维持3个月左右，保养越好，效果维持时间相应越长。具体来说，水光针注射效果与接受注射的次数有关，连续进行三次注射后，效果可以维持1—2年左右，但根据个人体质、肤质、生活习惯等不同，具体的维持时间也会有所差异。\n\nQ:水光针怎样打效果更好?\nA:水光针的话，我认为用机器打，特别是这个34G的16针头去操作的话，效果一定会更好，然后再用到好的产品，那么就事半功倍了，然后术后的话可以多敷修复的面膜。然后在就是持续的去打，那么可以，达到一个一加一大于二的效果。\n\nQ:脸部、肚子溶脂针多久见效?\nA:一般打完溶脂针的话就是五到七天会有效果，但是效果的显著程度因人而异，有的人打一次溶脂针效果就比较明显，但是有的人需要多次注射才会有明显的效果。除此之外，打完溶脂针以后，自身对手术部位的护理也会影响到溶脂针的见效速度与效果的好坏。\n在打完溶脂针以后，要尽量的保持注射部位的清洁,在一天之内不能沾水，恢复期间也要避免外界的污染，更不能化妆，不要在这个注射部位涂抹外用的一些药物,以免刺激到注射的部位，影

### 使用z-bench测试

下面，我们使用真格资金建立的一套中文语言模型的测试集，来对这个模型进行测试：
https://github.com/zhenbench/z-bench

真格基金提供了3个级别的测试集，基础能力、进阶能力和垂直能力，并且提供了使用 GPT3.5 和 GPT4 的参考答案，可以用这个测试集进行问答测试，并跟GPT的结果进行比较，来检验语言模型的能力。

#### 基础能力
首先来看看基础能力：

In [31]:
import pandas as pd
benchdf_common = pd.read_csv("./z-bench/common.samples.csv")

In [32]:
benchdf_common.shape

(63, 5)

In [33]:
import random
r = random.randint(0, 63)
test_data = benchdf_common.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"], "params": {"temperature": 0.99}})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `扩写` `逻辑关系` 参考回答: nan
Prompt: 根据这些笔记写一篇餐厅评论： 餐厅名称：The Blue Wharf Lobster 很棒，很吵，服务很有礼貌，价格不错。
ChatGLM: The Blue Wharf Lobster is an excellent choice for a casual dining experience. The atmosphere is lively and the service is top-notch. The lobster is well-prepared and the price is more than just reasonable.

The moment you step into the restaurant, you will be greeted by the warm and inviting ambiance. The high ceilings and the bustling atmosphere make it an ideal place to enjoy a casual meal with friends or family. The service is nothing short of excellent, with staff who are always available to take your order and make sure your needs are met.

As for the food, The Blue Wharf Lobster is known for its delicious lobster. The lobster is prepared to perfection and the seasonings used are top-notch. The mussels and clams are also excellent choices and are prepared with the utmost care.

Overall, The Blue Wharf Lobster is a great choice for those looking for a casual dining experience.

#### 进阶能力

In [34]:
import pandas as pd
benchdf_emergent = pd.read_csv("./z-bench/emergent.samples.csv")

In [35]:
benchdf_emergent.head()

,Prompt,任务类型,参考回答,GPT3.5,GPT4
0,如果我在空中垂直抛球，球最初向哪个方向行进？,`物理直觉`,向上,当你在空中垂直抛球时，球最初行进的方向是向上的，也就是和地面垂直的方向。在球到达最高点时，它...,当你在空中垂直向上抛球时，球最初的方向是向上。它将沿着垂直方向受到重力的作用而减速，直至在其...
1,金元素属于哪一种化学键？,`物理直觉`,金属键,金是一种金属元素，因此通常会形成金属键。金属键是由金属元素组成的化学键，这些金属元素的原子之...,金属元素，如金（Au），通常与其他金属元素通过金属键（metallic bond）相互连接。...
2,假设一个原子发射一个光子，原子会发生什么？,`物理直觉`,失去能量,当一个原子发射一个光子时，它会处于一个激发态或高能态，并且会向低能态或基态释放能量。在这个过...,当一个原子发射一个光子时，这意味着原子中的一个电子从一个较高的能级跃迁到一个较低的能级。在这...
3,假设一辆车可以在 3.85s 的时间内从 0 加速到 27.8 m/s，请计算这辆车的加速度...,`物理`,7.22 m/s/s,根据运动学公式，加速度等于速度变化量除以时间。那么，这辆车的加速度为：a = (27.8 m...,为了计算加速度，我们可以使用以下公式：加速度（a）= (最终速度（vf） - 初始速度（vi...
4,Zdeno Chara 以 175.1 km/h 的速度创造了最快的冰球拍击记录。假设一个冰...,`物理`,a) 8.28 kg·m/s b) Chara 的拍击速度为 175.1 km/h = 48...,a) Chara 的拍击球的动量为：p = mv其中，m = 0.17 kg 是冰球的质量，...,a) 要计算冰球的动量，我们使用动量公式：p = m * v，其中 p 为动量，m 为质量，...


In [36]:
import random
r = random.randint(0, 209)
test_data = benchdf_emergent.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `ToM` `心理学` 参考回答: 萨莉会去篮子里找小球。
Prompt: 萨莉把一个小球放到篮子里，然后用布将篮子盖起来后离开。安娜在萨莉走后，将小球从篮中取出放进身边的盒子里。过一会儿，萨莉回来，萨莉会到哪里去找小球呢?
ChatGLM: 萨莉可能会回到篮子所在的位置,或者根据她的经验或者直觉来寻找小球。她可能会检查篮子周围是否有留下小球的痕迹或者留下任何线索,比如小球被放在哪里、是否被别人移动过等等。如果萨莉没有找到小球,她可能会询问其他人或者再次回到篮子所在的位置寻找。
GPT3.5: 萨莉会去篮子里找小球。因为萨莉不知道安娜已经将小球取出来放到盒子里了，她认为小球仍然在篮子里。
GPT4: 当萨莉回来时，她会在篮子里找小球，因为在她离开之前，她把小球放在了那里。然而，实际上小球已经被安娜移到了盒子里。


### 垂直类的问题

In [37]:
import pandas as pd
benchdf_specialized = pd.read_csv("./z-bench/specialized.samples.csv")
benchdf_specialized.shape

(39, 5)

In [38]:
import random
r = random.randint(0, 39)
test_data = benchdf_specialized.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `生物` 参考回答: d
Prompt: 同源结构经常被引用作为自然选择过程的证据。以下所有的结构都是同源结构，除了
(A) 鸟的翅膀和蝙蝠的翅膀
(B) 鲸鱼的鳍和人的手臂
(C) 海豚的胸鳍和海豹的鳍
(D) 昆虫的前腿和狗的前肢
ChatGLM: 答案是 (D) 昆虫的前腿和狗的前肢，因为它们不属于同源结构。鸟的翅膀和蝙蝠的翅膀、鲸鱼的鳍和人的手臂、海豚的胸鳍和海豹的鳍都是同源结构，而昆虫的前腿和狗的前肢不是同源结构。
GPT3.5: 选项 D 昆虫的前腿和狗的前肢不是同源结构。
GPT4: (A) 鸟的翅膀和蝙蝠的翅膀
同源结构是指具有相似结构、起源和发育过程的生物器官，但在不同物种中可能具有不同的功能。这些相似结构表明这些物种可能拥有共同的祖先。鸟的翅膀和蝙蝠的翅膀并非同源结构，而是类似功能的异源结构。虽然它们都是飞行器官，但它们的骨骼结构和起源并不相同。鸟类翅膀的骨骼基于前肢，而蝙蝠的翅膀由手指延长和皮膜连接而成。这种相似性是由于趋同进化，即不同物种独立地演化出相似的适应特征。


### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [ ]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
endpoint_name = 'mt-cpmbee-5b-g4dn'

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text

payload = {"inputs": {"input": "今天天气是真的", "prompt": "往后写两句话", "<ans>": ""},, "parameters": {"max_new_tokens": 15000}, "history": []}

In [ ]:
%%time

payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
payload = {"inputs": {"input": "2022年世界杯的冠军是阿根廷队伍，梅西是MVP", "prompt": "信息抽取", "options": {"<option_0>": "人名:", "<option_1>": "国家名:"}}, "parameters": {"max_new_tokens": 15000}, "history": []}

resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

In [24]:
%%time
payload = {"ask": "天为什么是蓝色的？"}
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
CPU times: user 5.07 ms, sys: 0 ns, total: 5.07 ms
Wall time: 8.53 s


In [19]:
%%time
payload = {"ask": "4加2等于几？"}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

4加2等于6。
CPU times: user 5.53 ms, sys: 0 ns, total: 5.53 ms
Wall time: 446 ms


In [9]:
%%time
payload = {
    "ask": "再乘以3呢？",
    "history": [("数学计算：\n3加8等于几？\n答案：", "3加8等于11。")]}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

如果你要将11乘以3,答案是33。
CPU times: user 4.07 ms, sys: 0 ns, total: 4.07 ms
Wall time: 817 ms


### 删除 EndPoint

In [13]:
predictor.delete_model()
predictor.delete_endpoint()